# Load and initialize

settings

In [ ]:
# ubunto:


# home:
# plot dir
dir_plots = 'C:/Users/yishai/Desktop/biology/breast_cancer/Plots'

# plots:
    # plot dir
dir_plots = 'Plots/'
    # create visualization
visualize = True
    # show figures
show = True

# subsample = False

# sample number
j=str(3)


umap_iter = False
dbscan_iter = False
no_iter = False

n_samples = 20000


envirement

In [ ]:


#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 22 20:09:58 2021

@author: ronguy
"""
import time 
start = time.process_time()

import sys
from IPython.display import Image, display
from tqdm import tqdm_notebook,tqdm
# import time

import numpy as np
# import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
# import matplotlib.patches as  mpatches
plt.rcParams["figure.figsize"] = (5.0, 4.0)  # Set default size of plots.
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gray"

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 180) #according to screen width
# from pandas.core.base import PandasObject
# PandasObject.view = view #allows view meth pd.view

# from scipy import signal, stats
# import seaborn as sns

# from sklearn import metrics
from sklearn.cluster import KMeans, DBSCAN, MiniBatchKMeans
# from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
# from sklearn.metrics import silhouette_samples,silhouette_score
# from sklearn.neighbors import kneighbors_graph

# from lmfit import minimize, Parameters
from umap import UMAP

import scanpy as sc
import anndata

# import schist as scs

# import networkx as nx

# from castle.common import GraphDAG
# from castle.metrics import MetricsDAG
# from castle.datasets import IIDSimulation, DAG
# from castle.algorithms import PC,Notears,GOLEM,ANMNonlinear,DirectLiNGAM,ICALiNGAM,NotearsLowRank
# import notears.notears as notears

# from shapely.geometry import Point
# from shapely.geometry.polygon import Polygon

# import keras
# from keras import layers
# from keras.callbacks import EarlyStopping
# from keras.callbacks import ModelCheckpoint

# from statsmodels.graphics._regressionplots_doc import _plot_influence_doc
# from statsmodels.regression.linear_model import OLS
# from statsmodels.stats.outliers_influence import variance_inflation_factor



from usefull_functions import *
from functions import *
from normalization import *


%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload



load data; normalized (CD45-)  

In [ ]:
dict = pickle_load('k'+j+'_dict')
k = dict['k']

# downsample
original_k = k
k= subsample_k(original_k.copy(),n=n_samples) 
del  original_k



NamesAll = dict['names']['NamesAll']
ToNorm = dict['names']['ToNorm']
CellIden = dict['names']['CellIden']
EpiCols = dict['names']['EpiCols']
NMS = dict['names']['NMS']
all = dict['all']

del dict

Visualize data

no point to print data.describe() as it is whitened (std~1, mean~0) 

In [ ]:
for name in [NamesAll,ToNorm,CellIden,EpiCols,NMS]:
  
  print(name)
  print('\n')
# print(k.min().T)

# UMAP - CD 45 Negative

## Cell Identity

In [ ]:
min_dist=0.001
n_neighbors=60
t= f'min_dist = {min_dist}, n_neighbors = {n_neighbors}'
CAll=pd.concat([k]).copy()
X_2d,fig =draw_umap(CAll[CellIden],cc=CAll['H4'],min_dist=min_dist,n_neighbors=n_neighbors,rstate=42,title=t)
# plt.show()
fig.savefig('Plots/Tumor'+j+'_umap_CellIdentity.png')

# if umap_iter:

#     umap_params(k,names = CellIden,min_dist_range = 5,n_neighbors_range= 3)

# else:
#     X_2d,CAll= umap_best(k.copy(),names = CellIden,min_dist = 0.06, n_neighbors = 300)


In [ ]:
if show:
    plot_x2d(X_2d, NamesAll,CAll,show,
            title ='Tumor'+j+ ' Cell Iden UMAP', 
            figname = 'Tumor'+j+'_UMAP_CellIdentity_')

In [ ]:
    
min_samples=100
eps=0.2
t= f'min_samples = {min_samples}, eps = {eps}'
labels,fig=dbscan_plot(data = X_2d,eps=eps,min_samples=min_samples,title='t')
# fig.show()
fig.savefig('Plots/Tumor'+j+'_dbscan_CellIdentity.png')

# if not umap_iter:

#     if dbscan_iter:
#         dbscan_params(X_2d.copy(),eps_range = 8,min_samples_range=8)
#     else:
#         # label each sample from which cluster 
#         labels=dbscan_plot(X_2d.copy(),  eps = 0.26,min_samples = 262)


In [ ]:

# 

In [ ]:
# plt.figure()
# sns.histplot(k[labels!=0].CD45,color='r',element='step',fill=False,stat='density')
# sns.histplot(K.CD45,element='step',fill=False,stat='density')
# plt.yscale('symlog')

In [ ]:
K_clust = plotClusters(k,X_2d,labels,NamesAll,
             title = 'Clusters Tumor '+j+' - Cell Iden Based',figname = 'Tumor' +j+'_clusters_CellIden' )

corrMat(k,names = CellIden,title = 'T'+j+' Cell Iden Based',figname = 'Tumor' +j+'_corrMat_CellIden' )
corrMat(k,names = NMS,title = 'T'+j+' Cell Iden Based',figname = 'Tumor' +j+'_corrMat_CellIden(NMS)' )


In [ ]:

fig = MeanDist(k [k.Clust==0],k [k.Clust==1],CellIden)
fig.savefig('Plots/Tumor'+j+'_MeanDist_CellIdentity.png')



## Epigenetics Based

In [ ]:
MRK=EpiCols.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')

min_dist=0.001
n_neighbors=60
t= f'min_dist = {min_dist}, n_neighbors = {n_neighbors}'
CAll=pd.concat([k]).copy()
X_2d,fig =draw_umap(CAll[MRK],cc=CAll['H4'],min_dist=min_dist,n_neighbors=n_neighbors,rstate=42,title=t)
# plt.show()
fig.savefig('Plots/Tumor'+j+'_umap_Epigen.png')


# if umap_iter:

#     umap_params(k,names = MRK,min_dist_range = 5,n_neighbors_range= 3)

# else:
#     X_2d,CAll= umap_best(k.copy(),names = MRK,min_dist = 0.06, n_neighbors = 300)




In [ ]:

if show:
    plot_x2d(X_2d, NamesAll,CAll,show,
         title ='Tumor '+j+' Epigen UMAP', 
         figname = 'tumor'+j+'_UMAP_Epi_')

In [ ]:
min_samples=100
eps=0.2
t= f'min_samples = {min_samples}, eps = {eps}'
labels,fig=dbscan_plot(data = X_2d,eps=eps,min_samples=min_samples,title='t')
# fig.show()
fig.savefig('Plots/Tumor'+j+'_dbscan_Epigen.png')


# if not umap_iter:
#     if dbscan_iter:
#         dbscan_params(X_2d.copy(),eps_range = 8,min_samples_range=8)
#     else:
#         # label each sample from which cluster 
#         labels=dbscan_plot(X_2d.copy(),  eps = 0.26,min_samples = 262)


        

In [ ]:
K_clust = plotClusters(k,X_2d,labels,NamesAll,
             title = 'Clusters Tumor '+j+' - Epigen Based',figname = 'Tumor' +j+'_clusters_Epigen' )

corrMat(k,names = EpiCols,title = 'T'+j+' Epigen Based',figname = 'Tumor' +j+'_corrMat_Epigen' )




In [ ]:
fig = MeanDist(k [k.Clust==0],k [k.Clust==1],CellIden)
fig.savefig('Plots/Tumor'+j+'_MeanDist_Epigen.png')